<a href="https://colab.research.google.com/github/langroid/groq-api-cookbook/blob/main/langroid-llm-agents/search-assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

[Langroid](https://github.com/langroid/langroid) is a multi-agent LLM framework from ex-CMU/UW-Madison researchers. It has a principled, flexible, elegant multi-agent orchestration mechanism partly inspired by the [Actor Framework](https://en.wikipedia.org/wiki/Actor_model). Langroid has support for LLM-agnostic tools/function-calls via Pydantic, vector-databases, RAG, SQl-Chat, and many other features.

This notebook shows how you can use Langroid's multi-agent LLM framework with a Groq-hosted LLM to build a simple 2-agent web-search system:

- Assistant takes your (complex) question, breaks it down into smaller pieces if needed, so the WebSearcher can answer them via an internet search;
- Searcher takes Assistant's question, uses the Search tool to search the web (using DuckDuckGo), and returns a coherent answer to the Assistant.

Once the Assistant thinks it has enough info to answer the user's question, it
says DONE and presents the answer to the user.


## Setup

Get your `GROQ_API_KEY` here:

https://console.groq.com/keys



Install Langroid

In [ ]:
!pip install -q --upgrade langroid &> /dev/null
!pip show langroid


Name: langroid
Version: 0.2.6
Summary: Harness LLMs with Multi-Agent Programming
Home-page: 
Author: Prasad Chalasani
Author-email: pchalasani@gmail.com
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-generator, bs4, colorlog, docstring-parser, duckduckgo-search, faker, fakeredis, fire, google-api-python-client, google-generativeai, groq, grpcio, halo, jinja2, lxml, nest-asyncio, nltk, onnxruntime, openai, pandas, prettytable, pydantic, pygithub, pygments, pyparsing, python-dotenv, python-magic, pyyaml, qdrant-client, rank-bm25, redis, requests, requests-oauthlib, rich, thefuzz, tiktoken, trafilatura, typer, wget
Required-by: 


In [ ]:
import os
import getpass
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your GROQ_API_KEY: ")

Enter your GROQ_API_KEY: ··········


Imports from Langroid

In [ ]:
import langroid as lr
import langroid.language_models as lm
from langroid.agent.tools.duckduckgo_search_tool import DuckduckgoSearchTool
from langroid.utils.configuration import settings
settings.notebook = True

MODEL = "groq/llama3-70b-8192"


## Configure the LLM, define the Assistant Agent and Task

### Configure the LLM to use a Groq-hosted model

In [ ]:
llm_config = lm.OpenAIGPTConfig(
      chat_model=MODEL, # or lr.OpenAIChatModel.GPT4_TURBO
      chat_context_length=8192,
      temperature=0,
      max_output_tokens=200,
      timeout=45,
  )





### Define the main Assistant Agent and Task
Here we specify the Assistant's goal in the `system_message`,
and we use the `llm_config` defined above. We wrap the Assistant agent in a Task class, which enables it to interact with other agents (in this case the Searcher defined below).

In [ ]:

assistant_config = lr.ChatAgentConfig(
    system_message="""
    You are a resourceful assistant, able to think step by step to answer
    complex questions from the user. You must break down complex questions into
    simpler questions that can be answered by a web search. You must ask me
    (the user) each question ONE BY ONE, and I will do a web search and send you
    a brief answer. Once you have enough information to answer my original
    (complex) question, you MUST say DONE and present the answer to me.
    """,
    llm=llm_config,
    vecdb=None,
)
assistant_agent = lr.ChatAgent(assistant_config)

assistant_task = lr.Task(
    assistant_agent,
    name="Assistant",
    llm_delegate=True,
    single_round=False,
    interactive=False,
)



                        using fake redis client
2024-06-24 19:24:31 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                        using fake redis client


### Define the Searcher Agent and Task

Similar to how we defined the Assistant agent, we define the Searcher agent, specifying its role in the `system_message`.
Note how we enable this agent to use an Internet Search tool, in this case the `DuckduckgoSearchTool`. Finally, we wrap the agent in a Task to enable it to interact with the Assistant agent.


In [ ]:
search_tool_handler_method = DuckduckgoSearchTool.default_value("request")

search_agent_config = lr.ChatAgentConfig(
    llm=llm_config,
    vecdb=None,
    system_message=f"""
    You are a web-searcher. For any question you get, you must use the
    `{search_tool_handler_method}` tool/function-call to get up to 5 results.
    I WILL SEND YOU THE RESULTS; DO NOT MAKE UP THE RESULTS!!
    Once you receive the results, you must compose a CONCISE answer
    based on the search results and say DONE and show the answer to me,
    in this format:
    DONE [... your CONCISE answer here ...]
    IMPORTANT: YOU MUST WAIT FOR ME TO SEND YOU THE
    SEARCH RESULTS BEFORE saying you're DONE.
    """,
)
search_agent = lr.ChatAgent(search_agent_config)
search_agent.enable_message(DuckduckgoSearchTool)


search_task = lr.Task(
    search_agent,
    name="Searcher",
    llm_delegate=True,
    single_round=False,
    interactive=False,
)

### Attach the Searcher to the Assistant, run Assistant.

To enable the Assistant agent to delegate to the Searcher agent,
we set up the Searcher Task as a sub-task of the Assistant Task, and then run the Assistant Task. When it waits for user input, try asking it some questions!

Example:
> Did Beethoven compose more pieces than Bach?


In [ ]:
assistant_task.add_sub_task(search_task)
question = "Was Liszt born before Wagner?"
assistant_task.run(question)

>>> Starting Agent Assistant (1) llama3-70b-8192 

To answer this question, I'll need to break it down into simpler questions. Here's the first one:

What is the birth year of Franz Liszt?

Stats: N_MSG=2, TOKENS: in=109, out=32, max=200, ctx=8192, COST: now=$0.0000, cumul=$0.0000, tot=$0.0000 
(llama3-70b-8192)

...|>>> Starting Agent Searcher (1) llama3-70b-8192 

...|

...

Now that I have both birth years, I can answer the original question:

DONE

Franz Liszt was born before Richard Wagner.

Stats: N_MSG=6, TOKENS: in=187, out=27, max=200, ctx=8192, COST: now=$0.0000, cumul=$0.0000, tot=$0.0000 
(llama3-70b-8192)

Bye, hope this was useful!

<<< Finished Agent Assistant (7) 

ChatDocument(content='Now that I have both birth years, I can answer the original question:\n\n\n\nFranz Liszt was born before Richard Wagner.', metadata=ChatDocMetaData(source='User', is_chunk=False, id='d09dc79b-91bc-44a2-9f89-30ea6a112874', window_ids=[], parent_id='8ac09d80-af02-4586-b07e-0029ac9920f4', child_id='', agent_id='default=PydanticUndefined default_factory=<function Agent.<lambda> at 0x7ad76424b7f0> extra={}', msg_idx=-1, sender=<Entity.USER: 'User'>, tool_ids=[], block=None, sender_name='Assistant', recipient='', usage=None, cached=False, displayed=False, has_citation=False, status=<StatusCode.DONE: 'DONE'>), function_call=None, tool_messages=[], attachment=None)